In [ ]:
# Exploratory Data Analysis for "Državni Posao" Show

## Importing Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import warnings
warnings.filterwarnings('ignore')

# Setting plot aesthetics
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

## Data Loading and Initial Inspection

# Load the data (replace 'drzavni_posao.csv' with your actual file path)
df = pd.read_csv('drzavni_posao.csv')

# Display basic information
print(f"Dataset Shape: {df.shape}")
print("\nFirst 5 rows:")
display(df.head())

print("\nData types:")
display(df.dtypes)

print("\nSummary statistics for numerical columns:")
display(df.describe())

print("\nMissing values by column:")
display(df.isnull().sum())

## Data Cleaning and Preprocessing

# Function to extract duration in seconds from length column
def convert_length_to_seconds(length):
    if pd.isna(length):
        return np.nan
    
    try:
        parts = str(length).split(':')
        if len(parts) == 2:  # MM:SS format
            return int(parts[0]) * 60 + int(parts[1])
        elif len(parts) == 3:  # HH:MM:SS format
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        else:
            return np.nan
    except:
        return np.nan

# Convert length to seconds
df['length_seconds'] = df['length'].apply(convert_length_to_seconds)

# Convert date to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y.', errors='coerce')

# Extract episode number from title where missing
def extract_episode_number(title):
    if pd.isna(title):
        return np.nan
    
    match = re.search(r'Ep\.(\d+)', str(title))
    if match:
        return int(match.group(1))
    return np.nan

# Fill missing episode numbers
if df['episode_number'].isnull().any():
    df['episode_number'] = df.apply(
        lambda x: extract_episode_number(x['title']) if pd.isna(x['episode_number']) else x['episode_number'], 
        axis=1
    )

# Convert season and episode_number to integers where possible
df['season'] = pd.to_numeric(df['season'], errors='coerce')

# Create a category for special episodes
df['is_special'] = df['title'].str.contains('specijal|proslava|atmosfera|jubilej', case=False, regex=True, na=False)

# Check processed data
print("\nAfter preprocessing, first 5 rows:")
display(df.head())

## Exploratory Data Analysis

### 1. Basic Statistics by Season

# Count episodes per season
season_counts = df['season'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
ax = sns.barplot(x=season_counts.index, y=season_counts.values)
ax.bar_label(ax.containers[0])
plt.title('Number of Episodes per Season', fontsize=14)
plt.xlabel('Season', fontsize=12)
plt.ylabel('Number of Episodes', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()







In [ ]:
### 2. Episode Length Analysis

# Average episode length by season
plt.figure(figsize=(12, 6))
season_length = df.groupby('season')['length_seconds'].mean() / 60  # Convert to minutes
ax = sns.barplot(x=season_length.index, y=season_length.values)
ax.bar_label(ax.containers[0], fmt='%.1f')
plt.title('Average Episode Length by Season (Minutes)', fontsize=14)
plt.xlabel('Season', fontsize=12)
plt.ylabel('Average Length (minutes)', fontsize=12)
plt.tight_layout()
plt.show()

# Distribution of episode lengths
plt.figure(figsize=(14, 6))
sns.histplot(df['length_seconds']/60, bins=30, kde=True)
plt.title('Distribution of Episode Lengths', fontsize=14)
plt.xlabel('Length (minutes)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df['length_seconds'].mean()/60, color='red', linestyle='--', label=f'Mean: {df["length_seconds"].mean()/60:.2f} min')
plt.legend()
plt.tight_layout()
plt.show()

# Length of episodes over time
plt.figure(figsize=(16, 6))
# Filter out rows with missing dates for this plot
temp_df = df.dropna(subset=['date', 'length_seconds'])
temp_df = temp_df.sort_values('date')
plt.scatter(temp_df['date'], temp_df['length_seconds']/60, alpha=0.6)
plt.title('Episode Length Over Time', fontsize=14)
plt.xlabel('Release Date', fontsize=12)
plt.ylabel('Length (minutes)', fontsize=12)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
### 3. Release Schedule Analysis

# Episodes per month over time
if not df['date'].isnull().all():
    df_with_date = df.dropna(subset=['date'])
    df_with_date['year_month'] = df_with_date['date'].dt.to_period('M')
    monthly_counts = df_with_date.groupby('year_month').size()
    
    plt.figure(figsize=(16, 6))
    monthly_counts.plot(kind='bar')
    plt.title('Number of Episodes Released per Month', fontsize=14)
    plt.xlabel('Year-Month', fontsize=12)
    plt.ylabel('Number of Episodes', fontsize=12)
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

    # Average episodes per day of week
    df_with_date['day_of_week'] = df_with_date['date'].dt.day_name()
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = df_with_date['day_of_week'].value_counts().reindex(day_order)
    
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=day_counts.index, y=day_counts.values)
    ax.bar_label(ax.containers[0])
    plt.title('Number of Episodes by Day of Week', fontsize=14)
    plt.xlabel('Day of Week', fontsize=12)
    plt.ylabel('Number of Episodes', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()




In [ ]:
### 4. Special Episodes Analysis

# Proportion of special episodes
plt.figure(figsize=(10, 6))
special_counts = df['is_special'].value_counts()
plt.pie(special_counts, labels=['Regular', 'Special'], autopct='%1.1f%%', startangle=90, colors=['#5D9CEC', '#FC6E51'])
plt.title('Proportion of Special Episodes', fontsize=14)
plt.tight_layout()
plt.show()

# Special episodes by season
special_by_season = df.groupby('season')['is_special'].sum()
total_by_season = df.groupby('season').size()
special_percent = (special_by_season / total_by_season * 100)

plt.figure(figsize=(12, 6))
ax = sns.barplot(x=special_percent.index, y=special_percent.values)
ax.bar_label(ax.containers[0], fmt='%.1f%%')
plt.title('Percentage of Special Episodes by Season', fontsize=14)
plt.xlabel('Season', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:
### 5. Title Analysis (Word Cloud)

# If you have wordcloud installed
try:
    from wordcloud import WordCloud
    
    # Combine all titles
    all_titles = ' '.join(df['title'].dropna().astype(str))
    
    # Create and generate a word cloud image
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100, 
                          contour_width=3, contour_color='steelblue').generate(all_titles)
    
    # Display the generated image
    plt.figure(figsize=(14, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title('Word Cloud of Episode Titles', fontsize=14)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("WordCloud is not installed. To use this feature, run: pip install wordcloud")



In [ ]:
### 6. URL Analysis

# Video IDs from URLs
df['video_id'] = df['url'].str.extract(r'watch\?v=([^&]+)')

# Number of videos in different playlists
if 'index=' in str(df['url'].iloc[0]):
    df['playlist_index'] = df['url'].str.extract(r'index=(\d+)')
    df['playlist_index'] = pd.to_numeric(df['playlist_index'], errors='coerce')
    
    plt.figure(figsize=(14, 6))
    plt.scatter(df['playlist_index'], df['length_seconds']/60, alpha=0.6)
    plt.title('Episode Length by Playlist Position', fontsize=14)
    plt.xlabel('Position in Playlist', fontsize=12)
    plt.ylabel('Length (minutes)', fontsize=12)
    plt.tight_layout()
    plt.show()


In [ ]:
### 7. Correlation Analysis

# Create a correlation dataframe with numeric columns
corr_columns = ['season', 'length_seconds']
if 'episode_number' in df.columns and df['episode_number'].notna().any():
    corr_columns.append('episode_number')
if 'playlist_index' in df.columns:
    corr_columns.append('playlist_index')

# Only proceed if we have at least two columns for correlation
if len(corr_columns) >= 2:
    corr_df = df[corr_columns].dropna()
    
    # Compute correlation matrix
    corr_matrix = corr_df.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Correlation Matrix', fontsize=14)
    plt.tight_layout()
    plt.show()



In [ ]:
### 8. Time Series Analysis

# Episodes over time
if not df['date'].isnull().all():
    plt.figure(figsize=(16, 6))
    
    # Create a date range from min to max date
    df_date = df.dropna(subset=['date'])
    date_range = pd.date_range(start=df_date['date'].min(), end=df_date['date'].max(), freq='M')
    
    # Count episodes per month
    episodes_by_month = df_date.resample('M', on='date').size()
    
    # Plot
    plt.plot(episodes_by_month.index, episodes_by_month.values, marker='o', linestyle='-')
    plt.title('Number of Episodes Released Over Time', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Number of Episodes', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # Cumulative episodes over time
    plt.figure(figsize=(16, 6))
    cumulative = df_date.sort_values('date')
    cumulative['cumulative_count'] = range(1, len(cumulative) + 1)
    
    plt.plot(cumulative['date'], cumulative['cumulative_count'])
    plt.title('Cumulative Number of Episodes Over Time', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Total Episodes', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
## Advanced Analysis

### 9. Seasonality Analysis

if not df['date'].isnull().all():
    # Set up figure layout
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Episodes by month of year
    df_with_date = df.dropna(subset=['date'])
    df_with_date['month'] = df_with_date['date'].dt.month
    monthly_dist = df_with_date.groupby('month').size()
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Ensure we have all months (fill missing with 0)
    monthly_dist = monthly_dist.reindex(range(1, 13), fill_value=0)
    
    ax1.bar(range(1, 13), monthly_dist.values)
    ax1.set_xticks(range(1, 13))
    ax1.set_xticklabels(month_names)
    ax1.set_title('Number of Episodes by Month of Year', fontsize=14)
    ax1.set_ylabel('Number of Episodes', fontsize=12)
    
    # Episodes by year
    df_with_date['year'] = df_with_date['date'].dt.year
    yearly_dist = df_with_date.groupby('year').size()
    
    ax2.bar(yearly_dist.index, yearly_dist.values)
    ax2.set_title('Number of Episodes by Year', fontsize=14)
    ax2.set_xlabel('Year', fontsize=12)
    ax2.set_ylabel('Number of Episodes', fontsize=12)
    
    plt.tight_layout()
    plt.show()


In [ ]:
### 10. Episode Type Classification

# Creating broader categories based on title patterns
def categorize_episode(title):
    if pd.isna(title):
        return "Unknown"
    title = str(title).lower()
    
    if any(term in title for term in ['novogodišnji', 'nova godina']):
        return "New Year Special"
    elif any(term in title for term in ['božić', 'božićni']):
        return "Christmas Special"
    elif 'specijal' in title:
        return "Other Special"
    elif any(term in title for term in ['atmosfera', 'proslava', 'jubilej']):
        return "Behind the Scenes"
    elif 'intervju' in title:
        return "Interview"
    else:
        return "Regular Episode"

df['episode_category'] = df['title'].apply(categorize_episode)

# Visualize episode categories
cat_counts = df['episode_category'].value_counts()

plt.figure(figsize=(12, 6))
ax = sns.barplot(x=cat_counts.index, y=cat_counts.values)
ax.bar_label(ax.containers[0])
plt.title('Distribution of Episode Categories', fontsize=14)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Compare length by category
plt.figure(figsize=(14, 6))
sns.boxplot(x='episode_category', y='length_seconds', data=df, showfliers=False)
plt.title('Episode Length by Category', fontsize=14)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Length (seconds)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
### 11. Summary of Key Findings

print("\nSummary of Key Findings:")
print(f"- Total number of episodes: {len(df)}")
print(f"- Number of seasons: {df['season'].nunique()}")
print(f"- Average episode length: {df['length_seconds'].mean()/60:.2f} minutes")
print(f"- Shortest episode: {df['length_seconds'].min()/60:.2f} minutes")
print(f"- Longest episode: {df['length_seconds'].max()/60:.2f} minutes")
print(f"- Percentage of special episodes: {df['is_special'].mean()*100:.1f}%")

if not df['date'].isnull().all():
    date_range = df.dropna(subset=['date'])
    print(f"- First episode date: {date_range['date'].min().strftime('%Y-%m-%d')}")
    print(f"- Last episode date: {date_range['date'].max().strftime('%Y-%m-%d')}")
    print(f"- Total time span: {(date_range['date'].max() - date_range['date'].min()).days} days")

# Export clean data
df.to_csv('državni_posao_cleaned.csv', index=False)
print("\nCleaned data has been saved to 'državni_posao_cleaned.csv'")

# Display conclusions
print("\nKey Insights:")
print("1. [This will be filled based on actual data patterns]")
print("2. [This will be filled based on actual data patterns]")
print("3. [This will be filled based on actual data patterns]")